In [1]:
import os
from utils_finetune import create_model_wrapper
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import umap

from viz import generate_latent_space

In [2]:
trial_id =8
# 8 I imported umap and imported generate_latent_space from viz, turned off eval
main_dir = '/Users/jonaheaton/Desktop/stanford_umap_may29'
data_dir = os.path.join('/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/stanford-hmp2') # data_v1 is only the baseline data
output_dir = os.path.join(main_dir, 'latent_space_issues')
os.makedirs(output_dir, exist_ok=True)

In [3]:
model_dir = os.path.join(main_dir,'trainval_model')
encoder = create_model_wrapper(f'{model_dir}/encoder_info.json',f'{model_dir}/encoder_state.pt')
head = create_model_wrapper(f'{model_dir}/head_info.json',f'{model_dir}/head_state.pt',is_encoder=False)

Loaded model state from /Users/jonaheaton/Desktop/stanford_umap_may29/trainval_model/encoder_state.pt
Loaded model state from /Users/jonaheaton/Desktop/stanford_umap_may29/trainval_model/head_state.pt


In [4]:
X_train_val = pd.read_csv(os.path.join(data_dir, 'data_v3','X_stanford-hmp_trainval.csv'),index_col=0)
y_train_val = pd.read_csv(os.path.join(data_dir, 'data_v3','y_stanford-hmp_trainval.csv'),index_col=0)
X_test = pd.read_csv(os.path.join(data_dir, 'data_v3','X_stanford-hmp_test.csv'),index_col=0)
y_test = pd.read_csv(os.path.join(data_dir, 'data_v3','y_stanford-hmp_test.csv'),index_col=0)

In [5]:
# convert to X_train_val to tensor
X_train_val_tensor = torch.tensor(X_train_val.values, dtype=torch.float32)

In [6]:
# encoder.eval()
output = encoder.transform(X_train_val_tensor)

# convert to numpy
output = output.detach().numpy()
# save to csv
output_df = pd.DataFrame(output, index=X_train_val.index)
output_df.to_csv(os.path.join(output_dir, f'output_{trial_id}.csv'))


In [7]:
# def generate_latent_space(X_data, encoder, batch_size=128):
#     encoder.eval()
#     if isinstance(X_data, pd.DataFrame):
#         x_index = X_data.index
#         X_data = torch.tensor(X_data.to_numpy(), dtype=torch.float32)
#     Z = torch.tensor([])
#     with torch.inference_mode():
#         for i in range(0, len(X_data), batch_size):
#             # print(i, len(X_data))
#             Z_batch = encoder.transform(X_data[i:i+batch_size])
#             Z = torch.cat((Z, Z_batch.detach()), dim=0)
#         Z = Z.detach().numpy()
#     Z_df = pd.DataFrame(Z, index=x_index)
#     return Z_df

In [8]:
output_2_df = generate_latent_space(X_train_val, encoder)
output_2_df.to_csv(os.path.join(output_dir, f'output_opt2_{trial_id}.csv'))

In [9]:
# Check if there are any differences between output_2_df and output_df
differences = not output_2_df.equals(output_df)

if differences:
    print("There are differences between output_2_df and output_df")
else:
    print("There are no differences between output_2_df and output_df")

print('Maximum difference: ', np.max(np.abs(output_2_df.values - output_df.values)))

There are differences between output_2_df and output_df
Maximum difference:  21.293152


In [10]:
Z_trainval = generate_latent_space(X_train_val, encoder=encoder)
Z_trainval.to_csv(os.path.join(output_dir,f'Z_stanford-hmp_trainval_{trial_id}.csv'))

Z_test = generate_latent_space(X_test, encoder=encoder)
Z_test.to_csv(os.path.join(output_dir,f'Z_stanford-hmp_test_{trial_id}.csv'))